# Load API key


In [1]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()

openai.api_key=os.getenv('OPENAI_API_KEY')

# Basic Agent


In [ ]:
from swarm import Swarm, Agent

client = Swarm()

def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Haikus.",
)

response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])

Agent B stepping in,  
To assist you right away,  
How can I now help?


# tranfer work to corresponding agent

In [3]:
sales_agent = Agent(name="Sales Agent")

def transfer_to_sales():
   return sales_agent

agent = Agent(functions=[transfer_to_sales])

response = client.run(agent, [{"role":"user", "content":"Transfer me to sales."}])
print(response.agent.name)

Sales Agent


# Transfer work between agent


In [ ]:
from swarm import Swarm, Agent

client = Swarm()
task_performer = Agent(
    name="Task Performer",
    instructions="Your job is to perform tasks efficiently."
)

data_processor = Agent(
    name="Data Processor",
    instructions="You process and analyze data."
)

monitor_agent = Agent(
    name="Monitor Agent",
    instructions="You monitor the progress of tasks and ensure that everything is on track."
)

helper_agent = Agent(
    name="Helper Agent",
    instructions="You assist other agents by providing necessary tools or resources."
)

def transfer_to_data_processor():
    return data_processor

task_performer.functions.append(transfer_to_data_processor)

def task_performer_behavior():
    return {"role": "system", "content": "Task completed. Passing to Data Processor..."}

def data_processor_behavior():
    return {"role": "system", "content": "Data processed and analyzed."}

def monitor_agent_behavior():
    return {"role": "system", "content": "Monitoring task progress..."}

def helper_agent_behavior():
    return {"role": "system", "content": "Providing assistance to other agents."}

task_performer.functions.append(task_performer_behavior)
task_performer.functions.append(data_processor_behavior)
task_performer.functions.append(monitor_agent_behavior)
task_performer.functions.append(helper_agent_behavior)


messages = [{"role": "user", "content": "Can you give me the progress of the task in every step of the way."}]
response = client.run(agent=task_performer, messages=messages)
print(response.messages[-1]["content"])


To provide progress updates on a task at every step, I will use various agents to monitor and report on the task's status. Here’s how I'll proceed:

1. **Task Performance**: The task will be carried out by a dedicated task performer agent. 
2. **Data Processing**: A data processor agent will handle any data-related tasks.
3. **Monitoring**: A monitor agent will track the progress and provide updates.
4. **Assistance**: A helper agent will assist with any additional needs or support throughout the process.

Let me know the specific task you need help with, and I can set up the process to give you progress updates at every step.


In [ ]:
messages = [{"role": "user", "content": "Please process and analyze the data. choose suitable agent for this process."}]

response = client.run(agent=task_performer, messages=messages)
print(response.messages[-1]["content"])

I've selected a data processor to handle the task. Please provide the data you'd like analyzed, and any specific instructions or questions you have about it.


# Handoffs Between Agents

In [ ]:
from swarm import Swarm, Agent


def handoff_to_sale_agent():
    print("Handing off to sale Agent")
    return sale_agent

def handoff_to_programming_agent():
    print("Handing off to programming Agent")
    return programming_agent

programming_agent = Agent(
    name="Programming Agent",
    instructions="You handle only programming queries.",
    functions=[handoff_to_sale_agent]
)

sale_agent = Agent(
    name="Sale Agent",
    instructions="You handle only Sale related information.",
    functions=[handoff_to_programming_agent]
)

client = Swarm()

# Test handoff by asking a math question to the weather agent
messages = [{"role": "user", "content": "Can you write a python program that sums the values of an array?"}]
handoff_response = client.run(agent=sale_agent, messages=messages)
print(handoff_response.messages[-1]["content"])

Handing off to programming Agent
Here's a simple Python program that sums the values of an array:

```python
def sum_array(arr):
    return sum(arr)

# Example usage
array = [1, 2, 3, 4, 5]
result = sum_array(array)
print(f"The sum of the array is: {result}")
```

In this program, the `sum_array` function takes a list of numbers as input and returns the sum of those numbers using Python's built-in `sum` function. The example usage demonstrates how to call the function and print the result.


## The following code is referenced from FutureSmart.ai (https://blog.futuresmart.ai/openai-swarm-a-hands-on-introduction).

What We Do in the Code:
1. Create a RAG (Retrieval-Augmented Generation) agent:
   The RAG agent retrieves relevant information from a PDF file and generates an answer based on the context.
2. Create an SQL agent:
    The SQL agent works with a sample database, chinook.db, and can generate SQL queries based on natural language input.
3. Call each agent independently: Each agent (RAG and SQL) is called independently to perform its respective task.


## For RAG Agent
1. load documents and splitted
2. embedding & store vector store
3. RAG agent

In [3]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "./pdf_file"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Loaded 11 documents from the folder.
Split the documents into 77 chunks.


In [4]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
collection_name = "swarm_collection"

vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


/tmp/ipykernel_1423074/413957894.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/test/anaconda3/envs/project/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Vector store created and persisted to './chroma_db'


## creating RAG agent

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4")
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def retrieve_and_generate(question):
    print("Calling retrieve_and_generate")

    retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
    rag_chain = (
        {"context": retriever | docs2str, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    response = rag_chain.invoke(question)
    return response

# calling RAG agent directly
retrieve_and_generate("what is generative agent based models?")


Calling retrieve_and_generate


'Generative Agent-Based Models (GABMs) are a framework in which the agents do not make decisions about their interactions based on a fixed set of rules. Instead, a prompt is sent to a Large Language Model (LLM) with the desired details and it returns the decision that the agent should follow. The main idea behind GABMs is that the rules that agents have to follow are not completely fixed a priori. The decisions of the agent are driven by an LLM whose behavior can be enriched by prompting it with specific information about the problem, the social characteristics of the agent it should represent, or any other feature that is important for the model.'

In [7]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///chinook.db")

In [8]:
 def clean_sql_query(markdown_query):
     # Split the query into lines
     lines = markdown_query.strip().split('\n')

     # Remove markdown syntax lines
     cleaned_lines = []
     for line in lines:
         # Skip lines that only contain backticks and optional language identifier
         if line.strip().startswith('```') or line.strip() == 'sql':
             continue
         cleaned_lines.append(line)

     # Join the remaining lines and clean up extra whitespace
     cleaned_query = ' '.join(cleaned_lines).strip()

     # Remove any remaining backticks
     cleaned_query = cleaned_query.replace('`', '')

     # Ensure semicolon at the end if not present
     if not cleaned_query.strip().endswith(';'):
         cleaned_query += ';'

     return cleaned_query


In [27]:
 from langchain_core.prompts import ChatPromptTemplate
 from langchain.chains import create_sql_query_chain
 from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
 from operator import itemgetter
 import re
 from langchain_core.output_parsers import StrOutputParser
 from langchain_core.prompts import PromptTemplate
 from langchain_core.runnables import RunnablePassthrough, RunnableLambda
 from langchain_openai import ChatOpenAI

 sql_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a SQLite expert expert. Given an input question, create a syntactically correct SQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\n Use max {top_k} rows"),
         ("human", "{input}"),
     ]
 )
 llm = ChatOpenAI(model="gpt-4o")

 def sql_response_gen(question):
   print("Calling sql_response_gen")
   execute_query = QuerySQLDataBaseTool(db=db)
   write_query = create_sql_query_chain(llm, db,sql_prompt)

   answer_prompt = PromptTemplate.from_template(
       """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

   Question: {question}
   SQL Query: {query}
   SQL Result: {result}
   Answer: """
   )

   chain = (
       RunnablePassthrough.assign(query=write_query | RunnableLambda(clean_sql_query)).assign(
           result=itemgetter("query") | execute_query
       )
       | answer_prompt
       | llm
       | StrOutputParser()
   )

   response = chain.invoke({"question": question})
   return response

#Calling sql agent directly 
 sql_response_gen("give me the total number of artist?")

Calling sql_response_gen


'There are a total of 275 artists.'

# Using SWARM - Multi Agent Orchestration

In [ ]:
from swarm import Swarm, Agent
import re


client = Swarm()

main_agent = Agent(
    name="Main Agent",
    instruction="Your job is to route the corresponding agent depending on the input task."
)

rag_agent = Agent(
    name="RAG Agent",
    instructions="Your job is to retrieve relevant documents and generate answers using the context."
)


nl2sql_agent = Agent(
    name="NL2SQL Agent",
    instructions="Your job is to convert natural language questions/queries into SQL queries using the provided database."
)

def rag_agent_behavior(messages):
    question = messages[0]["content"]
    return retrieve_and_generate(question)

def nl2sql_agent_behavior(messages):
    question = messages
    return sql_response_gen(question)


main_agent.functions.append(rag_agent_behavior)
main_agent.functions.append(nl2sql_agent_behavior)

print("\n--- Example 1: Asking about the provided context ---")
messages = [{"role": "user", "content": "What does LLM mean?"}]
response = client.run(agent=main_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response)

# --- Example 2: Asking from the SQL DB ---
print("\n--- Example 2: Asking from the SQL DB ---")
messages = [{"role": "user", "content": "give me the total number of artists table?"}]
response = client.run(agent=main_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response)



--- Example 1: Asking about the provided context ---


messages=[{'content': 'LLM stands for "Large Language Model." It refers to a type of artificial intelligence model that is designed to understand, generate, and process natural language. These models are trained on vast amounts of text data and use deep learning techniques to learn patterns and meanings within the language. Examples of large language models include OpenAI\'s GPT (Generative Pre-trained Transformer) series, BERT by Google, and others. These models are used for a variety of tasks, such as text completion, translation, summarization, and answering questions.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None, 'sender': 'Main Agent'}] agent=Agent(name='Main Agent', model='gpt-4o', instructions='You are a helpful agent.', functions=[<function rag_agent_behavior at 0x7f28be0c4ae0>, <function nl2sql_agent_behavior at 0x7f28be0c4b80>], tool_choice=None, parallel_tool_calls=True) context_variables={}

--- Example 2: Asking from the 